In [22]:
import pandas as pd
from lxml import html
import numpy as np
from tqdm import tqdm
from matplotlib import pyplot as plt
from sklearn.decomposition import TruncatedSVD, NMF, PCA
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_distances
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.ensemble import RandomForestClassifier
import gensim
import numpy as np
from sklearn.cluster import MiniBatchKMeans
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from collections import Counter,defaultdict
from string import punctuation
from razdel import tokenize as razdel_tokenize
import os
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
%matplotlib inline

morph = MorphAnalyzer()
punct = punctuation+'«»—…“”*№–'
stops = set(stopwords.words('russian'))

# эти штуки нужны для ворд ту века, чтобы обучить модель на вики дате, вики дату надо запихать в эти функции
# а потом надо взять данные и их тоже запихать и тоооолько потом уже пытаться делать векторы и что то там себе оценивать!!!

def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0].normal_form for word in words if word and word not in stops]

    return ' '.join(words)

def tokenize(text):
    tokens = [token.text for token in list(razdel_tokenize(text))]
    tokens = [token for token in tokens if token.isalnum()]
    return ' '.join(tokens)

# вот эта штука нужна для уже предобученной модели русвек, прежде чем пихать в нее данные,
# их надо запихать в функцию normalize_mystem()
mapping = {}

for line in open('ru-rnc.map.txt'):
    ms, ud = line.strip('\n').split()
    mapping[ms] = ud
    
from pymystem3 import Mystem
m = Mystem()

def normalize_mystem(text):
    tokens = []
    norm_words = m.analyze(text)
    for norm_word in norm_words:
        if 'analysis' not in norm_word:
            continue
            
        if not len(norm_word['analysis']):
            lemma = norm_word['text']
            pos = 'UNKN'
        else:
            lemma = norm_word["analysis"][0]["lex"].lower().strip()
            pos = norm_word["analysis"][0]["gr"].split(',')[0]
            pos = pos.split('=')[0].strip()
        pos = mapping[pos]
        tokens.append(lemma+'_'+pos)
    return tokens


# тут будут лежать списки с близостями для каждой метрики
all_similarities = []

In [4]:
'''Преобразуйте тексты в векторы в каждой паре 5 методами  - SVD, NMF, Word2Vec (свой и  русвекторовский), 
Fastext. У вас должно получиться 5 пар векторов для каждой строчки в датасете.
Между векторами каждой пары вычислите косинусную близость (получится 5 чисел для каждой пары). 
Постройте обучающую выборку из этих близостей . Обучите любую модель (Логрег, Рандом форест или что-то ещё)
на этой выборке и оцените качество на кросс-валидации (используйте микросреднюю f1-меру). 
Попробуйте улучить метрику, изменив параметры в методах векторизации.
!!УТОЧНЕНИЕ: модель нужно обучить сразу на всех 5 близостях, а не по 1 модели на каждой близости!
SVD и NMF применяйте к данным напрямую, а w2w и fastext обучите на отдельном корпусе (как в первой части). '''

'Преобразуйте тексты в векторы в каждой паре 5 методами  - SVD, NMF, Word2Vec (свой и  русвекторовский), \nFastext. У вас должно получиться 5 пар векторов для каждой строчки в датасете.\nМежду векторами каждой пары вычислите косинусную близость (получится 5 чисел для каждой пары). \nПостройте обучающую выборку из этих близостей . Обучите любую модель (Логрег, Рандом форест или что-то ещё)\nна этой выборке и оцените качество на кросс-валидации (используйте микросреднюю f1-меру). \nПопробуйте улучить метрику, изменив параметры в методах векторизации.\n!!УТОЧНЕНИЕ: модель нужно обучить сразу на всех 5 близостях, а не по 1 модели на каждой близости!\nSVD и NMF применяйте к данным напрямую, а w2w и fastext обучите на отдельном корпусе (как в первой части). '

In [31]:
# Тут я буду обучать модель самостоятельно word2vec, не на парафразах! это точно не они
from gensim.models.word2vec import Word2Vec
from multiprocessing import cpu_count
import gensim.downloader as api
# Download dataset

file = open('wiki_data.txt').read().splitlines()
data_norm = [normalize(text) for text in file]
data_norm = [text for text in data_norm if text]
# Train Word2Vec model. Defaults result vector size = 100
# используй все ядра и отсеки слова с частотой 1
#model = Word2Vec([text.split() for text in data_norm], min_count = 10, workers=-1)
# FASTTEXT MODEL ON LEARNING
# fasttext = gensim.models.FastText([text.split() for text in data_norm], size=50, min_n=10, max_n=8) 
# Save and Load Model
# model.save('newmodel')
# model = Word2Vec.load('newmodel')

In [6]:
# теперь надо взять тексты, открываем парафразы
corpus_xml = html.fromstring(open('paraphrases.xml', 'rb').read())
texts_1 = []
texts_2 = []
classes = []

for p in corpus_xml.xpath('//paraphrase'):
    texts_1.append(p.xpath('./value[@name="text_1"]/text()')[0])
    texts_2.append(p.xpath('./value[@name="text_2"]/text()')[0])
    classes.append(p.xpath('./value[@name="class"]/text()')[0])
    
data = pd.DataFrame({'text_1':texts_1, 'text_2':texts_2, 'label':classes})

In [7]:
# word2vec
data['text_1_norm'] = data['text_1'].apply(normalize).apply(tokenize)
data['text_2_norm'] = data['text_2'].apply(normalize).apply(tokenize)

In [9]:
def get_embedding(text, model, dim):
    
    #text = text.split()
    #print(text)
    new_text = []
    for word in text:
        if word in model:
            new_text.append(word)
    # чтобы не доставать одно слово несколько раз
    # сделаем счетчик, а потом векторы домножим на частоту
    words = Counter(new_text)
    #print(words)
    total = len(new_text)
    vectors = np.zeros((len(words), dim))
    
    for i,word in enumerate(words):
        v = model[word]
        #print(v,'vvvvvv')
        vectors[i] = v*(words[word]/total) # просто умножаем вектор на частоту
        #print(vectors[i])
    
    if vectors.any():
        vector = np.average(vectors, axis=0)
    else:
        #print('else')
        vector = np.zeros((dim))
    
    return vector

In [12]:
# делаем для ворд ту века
# это размерность вектора
dim = 100
X_text_1_w2v = np.zeros((len(data['text_1_norm']), dim))
X_text_2_w2v = np.zeros((len(data['text_2_norm']), dim))
#print(model.wv.index2word)
for i, text in enumerate(data['text_1_norm'].values):
    X_text_1_w2v[i] = get_embedding(text, model, dim=100)
    #print( X_text_1_w2v[i],'X_text_1_w2v[i]')
for i, text in enumerate(data['text_2_norm'].values):
    X_text_2_w2v[i] = get_embedding(text, model, dim=100)

/home/alina/.local/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  import sys
/home/alina/.local/lib/python3.6/site-packages/ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [19]:
def cousine_sim_vectors(vec1, vec2):
    vec1 = vec1.reshape(1, -1)
    vec2 = vec2.reshape(1, -1)
    return cosine_similarity(vec1, vec2)[0][0]

In [102]:
similarities = {}
for i in range(len( X_text_1_w2v)):
    value = [cousine_sim_vectors(X_text_1_w2v[i], X_text_2_w2v[i])]
    similarities.setdefault(i,[]).extend(value)
print(similarities[0])

[0.8783001020339976]


In [103]:
# RUSVEC
rusvec = gensim.models.KeyedVectors.load_word2vec_format('180/model.bin', binary=True)

In [104]:
data['text_1_norm'] = data['text_1'].apply(normalize_mystem)
data['text_2_norm'] = data['text_2'].apply(normalize_mystem)

In [105]:
dim = 300
X_text_1_rusvec = np.zeros((len(data['text_1_norm']), dim))
X_text_2_rusvec = np.zeros((len(data['text_2_norm']), dim))

for i, text in enumerate(data['text_1_norm'].values):
    X_text_1_rusvec[i] = get_embedding(text, rusvec, dim=300)
    #print(X_text_1_rusvec[i])
    
    
for i, text in enumerate(data['text_2_norm'].values):
    X_text_2_rusvec[i] = get_embedding(text, rusvec, dim=300)

In [106]:
for i in range(len(X_text_1_rusvec)):
    value = [cousine_sim_vectors(X_text_1_rusvec[i], X_text_2_rusvec[i])]
    similarities.setdefault(i,[]).extend(value)
print(similarities[0])

[0.8783001020339976, 0.6844317556056889]


In [107]:
# FASTTEXT
data['text_1_norm'] = data['text_1'].apply(normalize).apply(tokenize)
data['text_2_norm'] = data['text_2'].apply(normalize).apply(tokenize)

In [108]:
dim = 100
X_text_1_fast = np.zeros((len(data['text_1_norm']), dim))
X_text_2_fast = np.zeros((len(data['text_2_norm']), dim))
#print(model.wv.index2word)
for i, text in enumerate(data['text_1_norm'].values):
    X_text_1_fast[i] = get_embedding(text, model, dim=100)
    #print( X_text_1_w2v[i],'X_text_1_w2v[i]')
for i, text in enumerate(data['text_2_norm'].values):
    X_text_2_fast[i] = get_embedding(text, model, dim=100)

/home/alina/.local/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  import sys
/home/alina/.local/lib/python3.6/site-packages/ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [109]:
for i in range(len(X_text_1_fast)):
    value = [cousine_sim_vectors(X_text_1_fast[i], X_text_2_fast[i])]
    similarities.setdefault(i,[]).extend(value)
print(similarities[0])

[0.8783001020339976, 0.6844317556056889, 0.8783001020339976]


In [110]:
# SVD
svd = TruncatedSVD(200)
tfidf = TfidfVectorizer(min_df=3, max_df=0.4, max_features=1000)
tfidf.fit(pd.concat([data['text_1_norm'], data['text_2_norm']]))
X_text_1 = svd.fit_transform(tfidf.transform(data['text_1_norm']))
X_text_2 = svd.fit_transform(tfidf.transform(data['text_2_norm']))

In [111]:
for i in range(len(X_text_2)):
    value = [cousine_sim_vectors(X_text_1[i], X_text_2[i])]
    similarities.setdefault(i,[]).extend(value)
print(similarities[0])

[0.8783001020339976, 0.6844317556056889, 0.8783001020339976, 0.07258040517513242]


In [66]:
# NMF
cv = CountVectorizer(min_df=3, max_df=0.4, max_features=1000)
X = cv.fit_transform(data_norm)
nmf = NMF(50)
nmf.fit(X)
X_text_1_nmf = nmf.transform(tfidf.transform(data['text_1_norm']))
X_text_2_nmf = nmf.transform(tfidf.transform(data['text_2_norm']))

/home/alina/.local/lib/python3.6/site-packages/sklearn/decomposition/_nmf.py:1077: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  " improve convergence." % max_iter, ConvergenceWarning)


In [112]:
for i in range(len(X_text_1_nmf)):
    value = [cousine_sim_vectors(X_text_1_nmf[i], X_text_2_nmf[i])]
    similarities.setdefault(i,[]).extend(value)
print(similarities[0])

[0.8783001020339976, 0.6844317556056889, 0.8783001020339976, 0.07258040517513242, 0.41675942861600385]


In [113]:
# делаем матрицу, чтобы запихать ее в форест
all_similarities = []
for value in similarities.values():
    all_similarities.append(value)
matrix_for_forest = np.array(all_similarities)
print(matrix_for_forest)

[[ 0.8783001   0.68443176  0.8783001   0.07258041  0.41675943]
 [ 0.93955153  0.7547428   0.93955153 -0.00107577  0.4892001 ]
 [ 0.9646989   0.8264399   0.9646989  -0.01766951  0.85280206]
 ...
 [ 0.8952867   0.52718837  0.8952867  -0.06267831  0.94976582]
 [ 0.88573977  0.44182585  0.88573977 -0.05209853  0.77623609]
 [ 0.6808095   0.53620972  0.6808095   0.00955381  0.07738828]]


In [114]:
# обучаем форест
y = data['label'].values
train_X, valid_X, train_y, valid_y = train_test_split(matrix_for_forest, y,random_state=1)
clf = RandomForestClassifier(n_estimators=100, max_depth=7, min_samples_leaf=15,
                             class_weight='balanced')
clf.fit(train_X, train_y)
preds = clf.predict(valid_X)
print(classification_report(valid_y, preds))

              precision    recall  f1-score   support

          -1       0.65      0.59      0.62       629
           0       0.50      0.53      0.51       737
           1       0.50      0.50      0.50       441

    accuracy                           0.55      1807
   macro avg       0.55      0.54      0.54      1807
weighted avg       0.55      0.55      0.55      1807



In [116]:
# оцениваем форест на кросс валидации
from sklearn.model_selection import cross_val_score
scores = cross_val_score(clf, train_X, train_y, cv=5)
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

0.55 accuracy with a standard deviation of 0.01
